In [1]:
import re
import os
import json
import statistics
import datetime
import warnings
from src.najdi_rok import najdi_rok
import polars as pl
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
df = pl.read_parquet(os.path.join("data","cnb_vyber.parquet"))
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))

In [55]:
rocni_pocty = df.filter(pl.col('rok').is_between(1901, 2023)).group_by('rok').len().sort('rok').rename({'len':'knih'}).with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))

In [35]:
rocni_pocty

rok,knih
datetime[μs],u32
1900-01-01 00:00:00,1355
1901-01-01 00:00:00,1807
1902-01-01 00:00:00,2038
1903-01-01 00:00:00,1973
1904-01-01 00:00:00,1896
…,…
2021-01-01 00:00:00,12935
2022-01-01 00:00:00,13273
2023-01-01 00:00:00,11749


In [21]:
def irozhlas_conf():

    return {
        "config" : {
             "title": {'font': "Noticia Text",
                      'fontSize': 14,
                      'anchor': 'start',
                       'fontWeight': 'bolder',
                    'subtitleFont': 'Noticia Text',
                      'subtitleFontSize': 12,
                       'subtitleFontWeight': 'lighter',
                      'subtitlePadding': 12,
                       'dy': -12,
                      },
             "axis": {
                "labelFont": "Asap",
                "titleFont": "Asap",
                 "fontWeight": "lighter",
                "titleFontWeight": "lighter",
                "labelFontSize": 10,
                "titleFontSize": 10,
                'labelPadding': 2,
                 'titlePadding': 10
            },
            "legend": {
                "labelFont": "Asap",
                "labelFontWeight": "normal",
                "titleFont": "Asap",
                "titleFontWeight": "normal",
                "labelFontSize": 10,
                "titleFontSize": 10
            }
        }
    }

alt.themes.register('irozhlas', irozhlas_conf)
alt.themes.enable('irozhlas')

ThemeRegistry.enable('irozhlas')

In [81]:
titulek = "Knižní produkce po letech"
podtitulek = ["Podíl jazyků na překladové literatuře – krásné, naučné i odborné."] #,"Angličtina a francouzština (téměř) zmizely jen během 2. světové války,","kdy dominovaly překlady z němčiny. Po osvobození raketově narostl podíl ruštiny."]
kredit = "zdroj dat: Česká národní bibliografie · vizualizace: iROZHLAS.cz · 2025"

In [113]:
alt.Chart(rocni_pocty.to_pandas(), title={'text': titulek, 'subtitle': podtitulek}).mark_bar(color="#7CA6E1").encode(
    alt.X('rok:T', title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    alt.Y('knih:Q', title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    alt.Color('knih:Q', alt.condition(('knih' > 2000), 
                                alt.ColorValue('red'), 
                                alt.ColorValue('black')))).properties(width=400, height=200)

TypeError: '>' not supported between instances of 'str' and 'int'

In [129]:
alt.Chart(rocni_pocty.to_pandas(), title={'text': titulek, 'subtitle': podtitulek}).mark_bar(color="#7CA6E1").encode(
    alt.X('rok:T', title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    alt.Y('knih:Q', title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    color=alt.condition(
        'datum.knih > 15000',  # Using datum to reference the field value
        alt.value('#84c0e4'),
        alt.value('#687fa9')
    )).properties(width=400, height=200)

alt.Chart(...)